**FUNCTION EXPLAINATION**

To make a robust fuction that can deal with any date format, use of both NLP approach(like text recognition, Machine Translation) as well as pattern matching(i.e. regular expressions) are to be leveraged which is on what lexnlp is built upon

**2 important points about get_date method of lexnlp module:**

It can detect any human readable date format! and returns an empty list if no date format is encountered.

It converts any date format into YYYY/MM/DD format thus makes subs. between 2 date objects possible.

function(DaysBtDates) parameters:

data : numpy ndarray/pandas dataframe

impute : takes imputation with mode into account if set True.

In [ ]:
# Installing required package
!pip install lexnlp

In [ ]:
def DaysBtDates(data, impute = False):
  
  # Converting data in any form into pandas dataframe
  data = pd.DataFrame(data)

  # Imputing with mode
  if impute == True:
    for i in range(data.shape[1]):
      data.iloc[:,i] =data.iloc[:,i].fillna(data.iloc[:,i].mode()[0])

  # Function to get column having dates in any possible format
  def DateColumnExtractor(data):

    # Creating empty list to store date column indexes
    date_col_index = []
    row_1 = data.iloc[1, :]

    # Extracting Date columns using NLP module
    for i,j in enumerate(row_1):
      
      # Getting dates from each data point as string
      text = str(j)
      k = list(lexnlp.extract.en.dates.get_dates(text))
      
      # lexnlp getdates method returns empty list if text is not a date
      if len(k) != 0:
        date_col_index.append(i)

    date_columns = data.iloc[:, date_col_index]
    return date_columns

  # This function converts all dates in the same format YYYY/MM/DD
  def FormatConversion():

    date_columns = DateColumnExtractor(data)

    date_formated = date_columns.copy()

    for i in range(date_columns.shape[0]):
      for j in range(date_columns.shape[1]):
        text = str(date_columns.iloc[i,j])
        k = list(lexnlp.extract.en.dates.get_dates(text))
        date_formated.iloc[i, j] = k[0]
    return date_columns,date_formated

  # This function returns a col having diff of dates bw two cols
  def date_difference(a,b):
   for k in range(a.shape[0]):
    d = [ abs((a.iloc[k,0] - b.iloc[k,0]).days) for k in range(a.shape[0])]
    return d  

  
  # Calling FormatConversion function
  date_cols, data_f = FormatConversion()

  # getting numbers of combinations of 2 from all date columns
  n = (data_f.shape[1]*(data_f.shape[1]-1))/2
  date_dif = np.zeros((data_f.shape[0],int(n)))   
  
  c = 0
  for i in range(data_f.shape[0]):
      for j in range(i+1, data_f.shape[1]):   
      
        a = pd.DataFrame(data_f.iloc[:, i])
        b = pd.DataFrame(data_f.iloc[:, j])

        date_dif[:, c] = date_difference(a,b)
        c = c+1
  return date_cols, pd.DataFrame(date_dif)     



# Importing Necessary modules
import lexnlp.extract.en.dates
import pandas as pd
import numpy as np   

# Test dataframe
list_of = [['raj', '123456', '88.00', '07/05/2020', '7th jan 1999', '0.234', '1999-07-02', '07-08-1997'],
           ['house', '123', '1288.00', '07/03/2020', '8th jan 1999', '0.234', '1998-07-02','07-09-1992'],
           ['rajesh rawat', '1236', '567.0456', '09/08/2020', '9th feb 1999', '0.234', '1996-07-02','07-09-1996']]
df_ = pd.DataFrame(list_of)
df_.iloc[0,6] = np.nan

# Calling Function
column_with_dates, final = DaysBtDates(df_, True)
print(final.head())
print(column_with_dates.head()) 